In [13]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir) 
import torch
from torch import nn
from torch.utils.data import DataLoader, RandomSampler, Dataset
from transformers.modeling_utils import (WEIGHTS_NAME, PretrainedConfig, PreTrainedModel,
                             SequenceSummary, PoolerAnswerClass, PoolerEndLogits, PoolerStartLogits)
from transformers import XLNetTokenizer, XLNetForSequenceClassification, XLNetPreTrainedModel, XLNetModel
from torch.nn import CrossEntropyLoss, BCEWithLogitsLoss
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from torch.utils.data.dataset import ConcatDataset
from XLNet import XLNetForMultiSequenceClassification, Dataset_multi, Dataset_3Way, get_predictions

import pandas as pd
import numpy as np
import random
from IPython.display import clear_output
from tqdm.notebook import tqdm, trange
import logging
import os
from dataclasses import dataclass
from enum import Enum
from typing import List, Optional, Union
import csv

In [2]:
@dataclass
class InputFeatures:

    input_ids: List[int]
    attention_mask: List[int]
    token_type_ids: Optional[List[int]] = None
    label_ids: Optional[List[int]] = None

In [29]:
from transformers import XLNetConfig

SyntaxError: invalid syntax (<ipython-input-29-52af46679745>, line 1)

In [30]:
model = XLNet

dict_keys(['xlnet-base-cased', 'xlnet-large-cased'])

In [16]:
@dataclass
class InputExample:
    
    uid: str
    text_a: str
    text_b: Optional[str] = None
    text_eval: Optional[str] = None
    label: Optional[str] = None
    multi_label: Optional[str] = None

    def to_json_string(self):
        """ Serialzes this instance to a Json string. """
        return json.dumps(dataclasses.asdict(self), indent=2) + "\n"

In [22]:
tokenizer.__dict__

{'_additional_special_tokens': ['<eop>', '<eod>'],
 '_bos_token': '<s>',
 '_cls_token': '<cls>',
 '_eos_token': '</s>',
 '_mask_token': '<mask>',
 '_pad_token': '<pad>',
 '_pad_token_type_id': 3,
 '_sep_token': '<sep>',
 '_unk_token': '<unk>',
 'added_tokens_decoder': {},
 'added_tokens_encoder': {},
 'do_lower_case': False,
 'init_inputs': (),
 'init_kwargs': {'vocab_file': 'C:\\Users\\cheny\\.cache\\torch\\transformers\\dad589d582573df0293448af5109cb6981ca77239ed314e15ca63b7b8a318ddd.8b10bd978b5d01c21303cc761fc9ecd464419b3bf921864a355ba807cfbfafa8'},
 'keep_accents': False,
 'max_len': 1000000000000,
 'max_len_sentences_pair': 999999999997,
 'max_len_single_sentence': 999999999998,
 'model_input_names': ['token_type_ids', 'attention_mask'],
 'padding_side': 'left',
 'remove_space': True,
 'sp_model': <sentencepiece.SentencePieceProcessor; proxy of <Swig Object of type 'sentencepiece::SentencePieceProcessor *' at 0x000001D7202C0EA0> >,
 'unique_added_tokens_encoder': {'</s>',
  '<cls>

In [3]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
trainset = Dataset_3Way("snli_train", tokenizer=tokenizer, three_tasks=False)
#train_sampler = RandomSampler(trainset)
#train_dataloader = DataLoader(trainset, sampler=train_sampler, batch_size=1)

In [5]:
batch_encoding = tokenizer.batch_encode_plus(
        [(data[0], data[1]) for data in trainset], max_length=128, pad_to_max_length=True,
)

In [7]:
features = []

for idx, data in enumerate(trainset):
    inputs = {k: batch_encoding[k][idx] for k in batch_encoding}
    
    feature = InputFeatures(**inputs, label_ids=data[2])
    features.append(feature)

In [31]:
def read_tsv(input_file, quotechar=None):
    """ Read a tab separated file """
    with open(input_file, "r", encoding="utf-8-sig") as f:
        return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

In [32]:
lines = read_tsv('../data/snli_train.tsv')

In [38]:
len(tokenizer.encode('text_b'))

5

In [36]:
lines[1]

['A person on a horse jumps over a broken down airplane.',
 'A person is training his horse for a competition.',
 '1']

In [39]:
max_len = max([len(tokenizer.encode(line[0])) for line in lines])

In [40]:
max_len

90